## Building A Chatbot
In this  I'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.



- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions



In [126]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [127]:
from langchain_groq import ChatGroq
model = ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x3175c4ce0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x3175c5bb0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [128]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my name is syam i am from chicago")])



AIMessage(content="Hi Syam, it's nice to meet you!  \n\nChicago is a great city. What part of Chicago are you from?  What do you like to do there?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 19, 'total_tokens': 59, 'completion_time': 0.072727273, 'prompt_time': 0.001329657, 'queue_time': 0.075292916, 'total_time': 0.07405693}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4ef2cecb-2a87-493a-9a41-6d285f2bb693-0', usage_metadata={'input_tokens': 19, 'output_tokens': 40, 'total_tokens': 59})

In [129]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is syam i am from chicago"),
        AIMessage(content="Hello Syam! It's nice to meet you. \n\nChicago is a great city. What's your favorite thing about living there?  \n\n"),
        HumanMessage(content="Hey What's my name and where i am from?")
    ]
)

AIMessage(content="You said your name is Syam and you are from Chicago! 😊  \n\nIs there anything else you'd like to talk about?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 74, 'total_tokens': 106, 'completion_time': 0.058181818, 'prompt_time': 0.002430934, 'queue_time': 0.065668499, 'total_time': 0.060612752}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3487ec46-dd5f-4447-9458-0107cd9530bd-0', usage_metadata={'input_tokens': 74, 'output_tokens': 32, 'total_tokens': 106})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [130]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [131]:
config = {"configurable": {"session_id": "chat1"}}

In [132]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is syam i am from chicago")],
    config=config
)

In [133]:
response.content

"Hi Syam, it's nice to meet you!  Chicago is a great city.  What part of Chicago are you from?  \n\nWhat can I do for you today?\n"

In [134]:
with_message_history.invoke(
    [HumanMessage(content="Hey what is my name ")],
    config=config
)

AIMessage(content='Your name is Syam.  \n\n😊  Is there anything else I can help you with? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 75, 'total_tokens': 99, 'completion_time': 0.043636364, 'prompt_time': 0.002368575, 'queue_time': 0.069517076, 'total_time': 0.046004939}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0390ed0b-ba80-4dc0-9aa4-a4589180eaab-0', usage_metadata={'input_tokens': 75, 'output_tokens': 24, 'total_tokens': 99})

In [135]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n\nWhat's your name?\n"

In [136]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Sawthi i am from India")],
    config=config1
)
response.content

"Hello Sawthi! It's nice to meet you. 👋\n\nI'm glad to know you're from India. Is there anything I can help you with today?  🇮🇳  😊  \n"

In [137]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is Sawthi!  😊 \n\nIt's great to chat with you, Sawthi. What can I do for you today? \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [138]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [139]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Syam")]})

AIMessage(content="Hi Syam! It's nice to meet you. 😊\n\nI'm happy to help answer any questions you have. Just ask away!  What's on your mind?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 32, 'total_tokens': 74, 'completion_time': 0.076363636, 'prompt_time': 0.001490746, 'queue_time': 0.065629389, 'total_time': 0.077854382}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e9cbb2b3-2b21-4e49-896f-1416daecf1a7-0', usage_metadata={'input_tokens': 32, 'output_tokens': 42, 'total_tokens': 74})

In [140]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [141]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is syam")],
    config=config
)

response

AIMessage(content="Hi Syam! It's nice to meet you. 😊 \n\nI'm here to help. What can I do for you? Just ask away, and I'll do my best to answer your questions!  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 32, 'total_tokens': 82, 'completion_time': 0.090909091, 'prompt_time': 0.001496267, 'queue_time': 0.067086838, 'total_time': 0.092405358}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4fe50d64-aa87-4831-ac36-b2d7bd0a362b-0', usage_metadata={'input_tokens': 32, 'output_tokens': 50, 'total_tokens': 82})

In [142]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Syam.  I remember! 😊  \n\nIs there anything else I can help you with?\n'

In [143]:
prompt = ChatPromptTemplate.from_messages(

    [

        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [144]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Syam")],"language":"Telugu"})
response.content

'నాకు తెలుసు, మీ పేరు శ్యాం. \n\nమీకు ఏమి సాయం చేయాలి? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [145]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [146]:
config = {"configurable": {"session_id": "chat4"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Syam")],"language":"Telugu"},
    config=config
)
response.content

'మీకు కల్యాణం, శ్యాం! మీకు ఏమి సహాయం చేయాలి? 😊 \n'

In [147]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)

In [148]:
response.content

'మీ పేరు శ్యాం అని తెలుసుకున్నాను. 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [152]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(

    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)


messages = [

    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Syam"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Strawberry ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="you're welcome!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]


trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like Strawberry ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content="you're welcome!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [159]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
 )

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences!  \n\nWhat's your favorite flavor? 😊🍦\n"

In [160]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2". 🧮  \n\n\n\nLet me know if you want to try another one! 😊\n'

In [162]:
## Lets wrap this in the MEssage History

with_message_history = RunnableWithMessageHistory(

    chain,
    get_session_history,
    input_messages_key="messages",
)

config={"configurable":{"session_id":"chat5"}}


In [163]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nWould you like to tell me your name? 😊  \n\n"

In [164]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"